In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.mlab as mlab
import math

In [2]:
np.random.seed(900)

In [3]:
def features_transform_basic(X):
    return np.hstack((np.ones((X.shape[0], 1)), X))

In [4]:
def features_transform_nonlinear(X):
    return np.column_stack((np.ones((X.shape[0], 1)), X, np.array(X[:,0]*X[:,1]).T, X*X))

In [5]:
def features_transform_f(X):
    return np.hstack((np.ones((X.shape[0], 1)), X*X))

In [6]:
def linear_regression(X, y):
    # Closed form solution to linear regression
    return np.linalg.pinv(X).dot(y)

In [7]:
def plot_dataset(D, y, w):
    # Get slope-intercept form (fline) out of the standard form (w)
    fline = lambda x: -(w[0] + w[1] * x) / w[2]
#     plt.plot([-1, 1], [fline(-1), fline(1)])
    plt.scatter(D[(y <= 0),1], D[(y <= 0),2], c='r')
    plt.scatter(D[(y > 0),1], D[(y > 0),2], c='g')
    plt.show()

In [8]:
def err(D, w, y_real):
    diff = np.sign(D.dot(w)) != y_real
    return np.mean(diff)

In [9]:
def add_noise(y, fraction = 0.1):
    idx = (np.random.random_sample((y.shape[0],)) <= 0.1)
    y[idx] = - y[idx]
    return y

In [10]:
def experiment(d, N_train, N_test, transform = features_transform_basic, h_w = None):
    # Generate train and test data
    D_train = np.random.uniform(-1, 1, (N_train, d))
    X_f_train = features_transform_f(D_train)
    D_test = np.random.uniform(-1, 1, (N_test, d))
    X_f_test = features_transform_f(D_test)
    # Weights for f given quadratic features
    w = np.array([-0.6, 1, 1])
    # Label the data
    y_train = add_noise(np.sign(X_f_train.dot(w)))
    y_test = add_noise(np.sign(X_f_test.dot(w)))
    
    # Train the model
    X_transformed_train = transform(D_train)
    X_transformed_test = transform(D_test)
    w_tr = linear_regression(X_transformed_train, y_train)
    # Plot the training set with the learned decision boundry
#     plot_dataset(D_train, y_train, w_tr)
    E_agree = None
    if h_w is not None:
        E_agree = err(X_transformed_test, h_w, np.sign(X_transformed_test.dot(w_tr)))

    # Calculate E_in and E_out
    E_in = err(X_transformed_train, w_tr, y_train)
    E_out = err(X_transformed_test, w_tr, y_test)
    return np.array([E_in, E_out, E_agree])

In [11]:
d = 2
R = 1000
N_test = 1000
N_train = 1000

In [12]:
# 2.8
E = np.array([experiment(d, N_train, N_test) for i in range(0, R)])
E = E[:,[0,1]]

E_in_avg, E_out_avg = np.mean(E, axis=0)
print("E_in =", E_in_avg)
print("E_out =", E_out_avg)

E_in = 0.504966
E_out = 0.520803


In [13]:
# 2.9 and 2.10
weight_list = [
    np.array([-1, -0.05, 0.08, 0.13, 1.5, 1.5]),
    np.array([-1, -0.05, 0.08, 0.13, 1.5, 15]),
    np.array([-1, -0.05, 0.08, 0.13, 15, 1.5]),
    np.array([-1, -1.5, 0.08, 0.13, 0.05, 0.05]),
    np.array([-1, -0.05, 0.08, 1.5, 0.15, 0.15])
]

for w_curr in weight_list:
    E = np.array([experiment(d, N_train, N_test, features_transform_nonlinear, w_curr) for i in range(0, R)])
    E_in_avg, E_out_avg, E_agree = np.mean(E, axis=0)
    print("E_in =", E_in_avg, ", E_out =", E_out_avg, ", E_agree =", E_agree)

E_in = 0.123807 , E_out = 0.12649 , E_agree = 0.037695
E_in = 0.123913 , E_out = 0.126345 , E_agree = 0.338427
E_in = 0.123724 , E_out = 0.126432 , E_agree = 0.338228
E_in = 0.123768 , E_out = 0.126201 , E_agree = 0.367616
E_in = 0.124198 , E_out = 0.127011 , E_agree = 0.438362
